In [ ]:
load_ext autoreload

In [ ]:
autoreload 2

In [ ]:
import numpy as np
import pyphi
import random
import makegridslibrary as me
import matplotlib
import matplotlib.pyplot as plt
import itertools

In [ ]:
C_BGR_N5=me.make_cmgrid_withbackground(5,3)
C_BGR_N5

In [ ]:
R_BGR_N5=me.make_random_withbackground(5,3)
R_BGR_N5

In [ ]:
# Make TPMs

# np.get_printoptions()
# np.set_printoptions(edgeitems=3)

# T_C_BGR_N9=me.make_tpm_withbackground(9,C_BGR_N9,3)

np.set_printoptions(edgeitems=16)
T_C_BGR_N5=me.make_tpm_withbackground(5,C_BGR_N5,2,.2,[])
T_C_BGR_N5_G=me.make_tpm_withbackground_gibs(5, C_BGR_N5, .2,[])

# fig = plt.figure()

T_R_BGR_N5=me.make_tpm_withbackground(5,R_BGR_N5,2)

# ax = fig.add_subplot(1,1,1)
# ax.set_aspect('equal')
# plt.imshow(T_BGR_N4, interpolation='nearest', cmap=plt.cm.PuBu)
# plt.colorbar()

# plt.show()

In [ ]:
T_C_BGR_N5_G

In [ ]:
# fix grid state or inputs and vary the rest : INTIALIZE

# Average distance for random initial states

# cm = C_BGR_N4
# N = 4
# thresh = 2
# initial_state = np.array([1,0,1,0,
#                          1,1,0,0])

# cm = C_BGR_N9
# N = int(cm.shape[0]/2)
# thresh = 3
# # intial state that lead to matching (very similar) : 1,1,1,0,0,0,0,1,1
# initial_state = np.array([1,1,1,0,0,0,0,1,1,
#                           1,1,0,0,0,0,1,1,1])

# cm = R_BGR_N5
cm = C_BGR_N5
N = int(cm.shape[0]/2)
thresh = 2

initial_state = np.array([0,0,0,1,0,
                          0,0,0,0,0])

iterations = 10
all_states = list()

nsim = 2**N
# nsim = 10000

# find_final_states = False
find_final_states = True

if find_final_states:
    
    permute = 1 # fixed background; vary initial
    final_states = list()
    
else:

    permute = 1 # fixed background; vary initial
#     permute = 2 # fixed initial; vary background
    
    distances = np.zeros(nsim)
    if permute == 1:
        good_grid_states = list()
    elif permute == 2:
        good_input_states = list()

# all_states = list()
# for isim in range(nsim):
    
#     this_state = np.random.randint(0, 2, N)

#     this_state = initial_state[0:N]
#     np.random.shuffle(this_state)

#     all_states.append(this_state)

In [ ]:
# fix grid state or inputs and vary the rest : RUN

all_states = itertools.product((0, 1), repeat=N)
for (isim, this_state) in enumerate(all_states):

    state = initial_state

    if permute == 1:
        state[0:N] = this_state
    elif permute == 2:
        state[N:] = this_state
    else:
        raise ValueError("Either permute initial state or background conditions")
        
    if nsim > 10:
        do_plot = False
    else:
        do_plot = True

    if find_final_states:
        final_state = me.running_behavior_conv(cm,N,iterations,state,thresh, do_plot=do_plot, do_print=False, return_final_state=True)
#         print(final_state)
        final_states.append(tuple(final_state))
    else:
        distances[isim] = me.running_behavior(cm,N,iterations,state,thresh, do_plot=do_plot, do_print=False)
        if distances[isim] == 0:
            if permute == 1:
                good_grid_states.append(this_state)
            elif permute == 2:
                good_input_states.append(this_state)

if find_final_states:
    final_states = {row for row in final_states if sum(row) > 0 and sum(row) < N}

In [ ]:
final_states

In [ ]:
# fix grid state or inputs and vary the rest : SUMMARY

# print("Mean distance: %f (chance %f)" % (np.mean(1*(distances==0)), 1/(sum(initial_state[0:N])*2**(N-sum(initial_state[0:N])))))

# there must be a better way of doing that, but it does the trick

# from itertools import permutations
# perms = list(set(permutations(initial_state[N:])))

perms = list(all_states)

chance_level = 1./isim

print("Mean distance: %f (chance %f)" % (np.mean(1*(distances==0)), chance_level))

if find_final_states:
    print("N unique final states %d (out of %d)" % (len(set(final_states)), nsim))
    print(final_states)
else:
    if permute == 1:
        print("N good grid states %d (out of %d)" % (len(good_grid_states), nsim))
    elif permute == 2:
        print("N good input states %d (out of %d)" % (len(good_input_states), nsim))

In [ ]:
good_grid_states

In [ ]:
[row for row in good_input_states if sum(row) > 0 and sum(row) < N]

In [ ]:
final_state[-1] = -1
final_state

In [ ]:
print(final_state)
from itertools import permutations
perms = list(set(permutations(final_state)))
perms

In [ ]:
good_grid_states

In [ ]:
good_input_states

In [ ]:
#test, for convervence using actual thresh
M=7
l=0.5
N=9
cm=C_BGR_N4
it=10
for i in range(M):
    nrfiring=int(l*N)
    nrfiringbackground=nrfiring
    [state,background]=me.pick_random_state(N,nrfiring,nrfiringbackground)
    me.running_behavior(cm,N,it,state,0, do_plot=False)

What we see is: 
- Some structured background conditions cause the GRID to match, but on with specific initial states (not very different from the background)
- No structured backgound caused the RANDOM to match, but vary few random background did it.

- Next we need to find the concepts in each one of the four conditions

In [ ]:
# Starting Pyphi

network = pyphi.Network(T_C_BGR_N5, connectivity_matrix=C_BGR_N5)

In [ ]:
grid_state = good_grid_states[0]
print(grid_state)

input_state = good_input_states[3]
print(input_state)

state = grid_state + input_state
state

In [ ]:
state = [0,0,0,0,0,0,0,0,0,0]
thegrid = pyphi.Subsystem(network, state, range(N))

In [ ]:
mip = pyphi.compute.big_mip(thegrid)

In [ ]:
print("{} ({} concepts)".format(mip.phi, len(mip.unpartitioned_constellation)))
print(mip.subsystem)
print(mip.cut)


In [ ]:
print(cm)

state1 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1])
print(state1)
thegrid = pyphi.Subsystem(network, state1, range(N))
mip1 = pyphi.compute.big_mip(thegrid)

state2 = np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1])
print(state2)
thegrid = pyphi.Subsystem(network, state2, range(N))
mip2 = pyphi.compute.big_mip(thegrid)

In [ ]:
mip1.partitioned_constellation

In [ ]:
print(mip1.cut)

for x in sorted(mip1.unpartitioned_constellation):
#     for y in x.mechanism:
#         print(y, '_', end='')
    print(x.mechanism, x.phi)

print('')

for x in sorted(mip1.partitioned_constellation):
#     for y in x.mechanism:
#         print(y, '_', end='')
    print(x.mechanism, x.phi)


In [ ]:
len(mip1.unpartitioned_constellation)

In [ ]:
bar_inputs = [ (0, 0, 0, 1, 1),
 (0, 0, 1, 1, 0),
 (0, 1, 1, 0, 0),
 (1, 0, 0, 0, 1)]
bar_inputs

In [ ]:
shuffled_inputs = [ (0, 0, 1, 0, 1),
 (1, 0, 0, 1, 0),
 (0, 1, 0, 0, 1),
 (1, 0, 1, 0, 0)]
shuffled_inputs

In [ ]:
do_plot = False

# try all possible states as initial state for the bar and the shuffled

bar_states = dict()
shuffled_states = dict()

for bar_input in bar_inputs:
    
    this_bar_states = list()
    all_states = itertools.product((0, 1), repeat=N)
    for this_state in all_states:
    
        state = this_state + bar_input
        converged_state = me.running_behavior_conv(cm,N,10,state,thresh,do_plot=do_plot, do_print=False, return_final_state=True)
        this_bar_states.append(tuple(converged_state))
    
    bar_states[bar_input] = {x for x in this_bar_states}

for shuffled_input in shuffled_inputs:
    
    this_shuffled_states = list()
    all_states = itertools.product((0, 1), repeat=N)
    for this_state in all_states:
    
        state = this_state + shuffled_input
        converged_state = me.running_behavior_conv(cm,N,10,state,thresh,do_plot=do_plot, do_print=False, return_final_state=True)
        this_shuffled_states.append(tuple(converged_state))
    
    shuffled_states[shuffled_input] = {x for x in this_shuffled_states}

In [ ]:
bar_states

In [ ]:
shuffled_states

In [ ]:
# do_plot = True
do_plot = False

cm = C_BGR_N5

results_bar = dict()
results_shuffled = dict()

for (ibar, bar_input) in enumerate(bar_states):
    
    print ('Bar {} out of {}'.format(ibar+1, len(bar_states)))

    all_states = itertools.product((0, 1), repeat=N)
    for (istate, this_state) in enumerate(all_states):
    
        bar_state = this_state + bar_input
        
        print ('\tState {} ({} out of {}) : '.format(bar_state, istate+1, 2**N), end='')

        try:
            thegrid = pyphi.Subsystem(network, bar_state, range(N))
            mip_bar = pyphi.compute.big_mip(thegrid)
            concepts_bar = [x.phi for x in mip_bar.unpartitioned_constellation]
            results_bar[bar_state] = (mip_bar.phi, concepts_bar)
            print ('{:1.5f} (Phi {:1.5f} * not weighted {:1.5f} ({} concepts))'.format(mip_bar.phi * sum(concepts_bar), 
                                                                       mip_bar.phi, sum(concepts_bar), 
                                                                        len(concepts_bar) ))
        except pyphi.exceptions.StateUnreachableError:
            print(' Invalid.')
    
for (ishuffled, shuffled_input) in enumerate(shuffled_states):
    
    print ('Shufled {} out of {}'.format(ishuffled+1, len(shuffled_states)))

    all_states = itertools.product((0, 1), repeat=N)
    for (istate, this_state) in enumerate(all_states):
    
        shuffled_state = this_state + shuffled_input
        
        print ('\tState {} ({} out of {}) : '.format(shuffled_state, istate+1, 2**N), end='')

        try:
            thegrid = pyphi.Subsystem(network, shuffled_state, range(N))
            mip_shuffled = pyphi.compute.big_mip(thegrid)
            concepts_shuffled = [x.phi for x in mip_shuffled.unpartitioned_constellation]
            results_shuffled[shuffled_state] = (mip_shuffled.phi, concepts_shuffled)
            print ('{:1.5f} (Phi {:1.5f} * not weighted {:1.5f} ({} concepts))'.format(mip_shuffled.phi * sum(concepts_shuffled), 
                                                                       mip_shuffled.phi, sum(concepts_shuffled), 
                                                                        len(concepts_shuffled) ))
        except pyphi.exceptions.StateUnreachableError:
            print(' Invalid.')

In [ ]:
matching_bar = np.zeros(len(results_bar))
for (ibar, bar_result) in enumerate(results_bar):
    
    matching_bar[ibar] = np.mean(np.array(results_bar[bar_result][1])*results_bar[bar_result][0])
#     matching_bar[ibar] = np.sum(np.array(results_bar[bar_result][1])*results_bar[bar_result][0])
    
matching_shuffled = np.zeros(len(results_shuffled))
for (ishuffled, shuffled_result) in enumerate(results_shuffled):

    matching_shuffled[ishuffled] = np.mean(np.array(results_shuffled[shuffled_result][1])*results_shuffled[shuffled_result][0])
#     matching_shuffled[ishuffled] = np.sum(np.array(results_shuffled[shuffled_result][1])*results_shuffled[shuffled_result][0])

print(matching_bar) 
print(matching_shuffled) 
# print('Matching : %2.5f' % (np.mean(matching_bar) - np.mean(matching_shuffled)))
print('Matching : %2.2f' % (np.sum(matching_bar) - np.sum(matching_shuffled)))

matching_bar = np.zeros(len(results_bar))
for (ibar, bar_result) in enumerate(results_bar):
    
    matching_bar[ibar] = np.mean(np.array(results_bar[bar_result][1]))
#     matching_bar[ibar] = np.sum(np.array(results_bar[bar_result][1]))
    
matching_shuffled = np.zeros(len(results_shuffled))
for (ishuffled, shuffled_result) in enumerate(results_shuffled):

    matching_shuffled[ishuffled] = np.mean(np.array(results_shuffled[shuffled_result][1]))
#     matching_shuffled[ishuffled] = np.sum(np.array(results_shuffled[shuffled_result][1]))

print(matching_bar) 
print(matching_shuffled) 

# print('Matching (not wieghted): %2.5f' % (np.mean(matching_bar) - np.mean(matching_shuffled)))    
print('Matching (not wieghted): %2.2f' % (np.sum(matching_bar) - np.sum(matching_shuffled)))

In [ ]:
constellation = mip_shuffled.unpartitioned_constellation
[concept.mechanism for concept in constellation]

In [ ]:
[concept.phi for concept in constellation]

In [ ]:
testgrid = pyphi.Subsystem(network, [1,0,0,0,0,0,0,0,0,0], range(N))
mip_test = pyphi.compute.big_mip(testgrid)
constellation = mip_test.unpartitioned_constellation
[concept.mechanism for concept in constellation]

In [ ]:
concepts = [concept.mechanism for concept in constellation]

In [ ]:
constellation[0].time

In [ ]:
constellation

In [ ]:
%connect_info